<a href="https://colab.research.google.com/github/Sebato/MLProject/blob/main/Projet_ML_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importation des différentes librairies utiles pour le notebook

!pip install langdetect 
!pip install contractions
!pip install wordcloud

#Sickit learn met régulièrement à jour des versions et 
#indique des futurs warnings. 
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# librairies générales
import pickle 
import pandas as pd
from scipy.stats import randint
import numpy as np
import string
import time
import base64
import re
import sys

import contractions

# librairie BeautifulSoup
from bs4 import BeautifulSoup

# librairie affichage
import matplotlib.pyplot as plt
import seaborn as sns
import wordcloud

## detection de language
import langdetect 

import nltk
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from nltk import RegexpParser
# il est possible de charger l'ensemble des librairies en une seule fois 
# décocher le commentaire de la ligne ci-dessous
#nltk.download('all') 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag
nltk.download('tagsets')
nltk.download("stopwords")
nltk.download('wordnet')

from nltk.corpus import stopwords

import spacy
from spacy.tokens import Span
# il faut sélectionner pour quelle langue les traitements vont être faits.




#Récupération des données


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

my_local_drive='/content/drive/MyDrive/ProjetML'
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive

%pwd

In [ ]:
nltk.download("stopwords")
from nltk.corpus import stopwords
import nltk
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

In [ ]:
import pandas as pd
df_train = pd.read_csv('data/HAI817_Projet_train.csv')
# df.drop_duplicates(subset=['title'], keep='first', inplace = True)
display(df_train)

In [ ]:
data = {
    "A": ["TeamA", "TeamB", "TeamB", "TeamC", "TeamA"],
    "B": [50, 40, 40, 30, 50],
    "C": [True, False, False, False, True]
}
  
df = pd.DataFrame(data)
display(df)
display(df.drop_duplicates())

In [ ]:
# display(dd)

# Préparation des données
- Tokenization et mise en lowercase
### Création de liste des mots qui apparaissent dans le texte et dans le titre de chaque document 

In [ ]:
def cleaningData(data):
 """etapes de nettoyage:
  - supprimer doublons
  - suppression de la fin des URLs
  - transformation des caractères numériques en mots
  - 
      
  """
  # regex = re.compile('(?u)\\b\\w\\w+\\b')
  # tokens = re.findall(regex, text.lower()) # notice lowercase=True param
  # vocab = sorted(set(tokens))
  # counts = Counter(tokens)
  # counts = [counts[key] for key in sorted(counts.keys())]
  # #vocab, counts = list(zip(*sorted(counter.items()))) # one liner with asterisk unpacking
  # print(pd.DataFrame([counts], columns = vocab)) 


In [ ]:
import re


#Prise en compte des éléments qui doivent être regroupés
regex_str = [
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    # r'(^($|£)[\d]+(,[\d]+)+(.[\d]{2})?)' # Monnaies
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # nombres
    r"(?:[a-z][a-z'\-_]+[a-z])", # mots avec - et '
    r'(?:[\w_]+){2,}' # autres mots
    #r'(?:\S)' # le reste
]

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)

# def tokenize(s):
#   tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
  
#   return tokens_re.findall(s)

def tokenize(s):
    ps = nltk.stem.porter.PorterStemmer()
    stop_words = list(set(stopwords.words("english")))
    tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
    tokens = tokens_re.findall(s)
    stems = [ps.stem(t) for t in tokens if t not in stop_words]
    return stems

    
tokenPattern = r'('+'|'.join(regex_str)+')'
print("\n--------\n"+tokenPattern)
    
# def preprocess(text):
#   """ etapes de pre processing :
#       - suppréssion de la fin des URLs
#       - transformation des caractères numériques en mots
#       - 
      
#   """
#   ps = nltk.stem.porter.PorterStemmer()
#   text = " ".join([ps.stem(w) for w in text.split()])
#   return text.lower()
  

In [ ]:
def get_tokens(dataframe):    
    full_texts = []
    for i in range(len(dataframe)):
        full_texts.append((str(dataframe.iloc[i].title) + " " + str(dataframe.iloc[i].text)))
    tokens = [t.split() for t in full_texts]
    tokens = [[t.lower() for t in lt] for lt in tokens]
    return tokens


In [ ]:
def elim_stopwords(words, stopwords):
    return [word for word in words if word not in stopwords]

    

In [ ]:
def stemming(dataframe):
    """ """
    ps=nltk.stem.porter.PorterStemmer()
    dataframe["tokens"].transform(lambda tokens: [ps.stem(word) for word in tokens])
    return dataframe


In [ ]:
def prepare_data(dataframe, classes):
    dataframe["tokens"] = get_tokens(dataframe)
    
    swords = set(stopwords.words("english"))
    dataframe["tokens"] = dataframe["tokens"].transform(lambda t: [word for word in t if word not in swords])
    
    #retirer émoticones
    

    ps=nltk.stem.porter.PorterStemmer()
    dataframe["tokens"] = dataframe["tokens"].transform(lambda tokens: [ps.stem(word) for word in tokens])

    dataframe = dataframe[dataframe["our rating"].isin(classes)]
    dataframe = dataframe.reset_index(drop=True)
        
    return dataframe

In [ ]:
#df_train = prepare_data(df_train, ["true", "false"])
#(display(df_train.head()))


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def create_bag(dataframe):
    corpus = []
    for i in range(len(dataframe)):
        corpus.append((str(dataframe.iloc[i].title) + " " + str(dataframe.iloc[i].text)))
    vectorizer = CountVectorizer(tokenizer=tokenize)
    # creation du vocabulaire
    vector = vectorizer.fit_transform(corpus)
    df = pd.DataFrame(data=vectorizer.transform(corpus).toarray(), columns=vectorizer.get_feature_names_out())
    df.insert(0, "our_rating", dataframe["our rating"])
    return (vectorizer, df)
    


In [ ]:
bag, bag_train = create_bag(df_train)
print(bag.vocabulary_)


In [ ]:
import pandas as pd

"""etapes de nettoyage:
  - Supression des lignes avec deux rating different
  - Supression des doublons
  - Supression des lignes qui n'ont pas texte et titre
"""
def cleaningData(df):
  df = pd.read_csv('data/HAI817_Projet_train.csv')
  df_merge = pd.merge(df, df, how='inner', left_on=['title', 'text'], right_on=['title', 'text'])
  df_supr = df_merge[(df_merge['our rating_x'] != df_merge['our rating_y'])]
  df_supr = df[df['public_id'].isin(df_supr['public_id_x'].drop_duplicates().values)]
  df.drop(df_supr.index, inplace = True)
  df.drop_duplicates(subset=['title'], keep="first", inplace = True)
  
  return df

In [ ]:
def intersect(bag_train, bag_test, df_train, df_test):
  vocab_train = list(bag_train.vocabulary_.keys())
  vocab_test = list(bag_test.vocabulary_.keys())

  inter = [mot for mot in vocab_train if mot in vocab_test]

  train_drop = [mot for mot in vocab_train if mot not in inter]
  test_drop =  [mot for mot in vocab_test  if mot not in inter]

  df_train = df_train.drop(train_drop, 1)
  df_test = df_test.drop(test_drop, 1)

  return (df_train, df_test)

# Cellule définition:
- Tokenizer
- Bag of words
- drop duplicates
###renvoie les données et leur classe pour application directe d'un modèle

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import re

#trouve les mots en communs entre les deux datasets et supprime les autres
def intersect(bag_train, bag_test, df_train, df_test):
  vocab_train = list(bag_train.vocabulary_.keys())
  vocab_test = list(bag_test.vocabulary_.keys())

  inter = [mot for mot in vocab_train if mot in vocab_test]

  # print(inter[:20])
  
  # train_drop = [mot for mot in vocab_train if mot not in inter]
  # test_drop = [mot for mot in vocab_test if mot not in inter]

  train_drop = []
  test_drop = []

  for mot in vocab_train:
    if mot not in inter:
      train_drop.append(mot)
 
  for mot in vocab_test:
    if mot not in inter:
      test_drop.append(mot)
  
  df_train = df_train.drop(train_drop, 1)
  df_test = df_test.drop(test_drop, 1)

  # display(df_train.head())
  # display(df_test.head())

  return (df_train, df_test)


#Mise en forme des données (suppression doublons, sans titres)
def cleaningData(df):
  df.rename(columns={ df.columns[0]: "public_id" }, inplace = True)
  df_merge = pd.merge(df, df, how='inner', left_on=['title', 'text'], right_on=['title', 'text'])
  df_supr = df_merge[(df_merge['our rating_x'] != df_merge['our rating_y'])]
  df_supr = df[df['public_id'].isin(df_supr['public_id_x'].drop_duplicates().values)]
  df.drop(df_supr.index, inplace = True)
  df.drop_duplicates(subset=['title'], keep="first", inplace = True)
  df.reset_index(inplace = True, drop = True)
  return df


#Prise en compte des éléments qui doivent être regroupés
regex_str = [
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    # r'(^($|£)[\d]+(,[\d]+)+(.[\d]{2})?)' # Monnaies
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # nombres
    r"(?:[a-z][a-z'\-_]+[a-z])", # mots avec - et '
    r'(?:[\w_]+){2,}' # autres mots
    #r'(?:\S)' # le reste
]

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)

#tokenizer amélioré (stopwords; regex; stems; )
def tokenize(s):
    ps = nltk.stem.porter.PorterStemmer()
    stop_words = list(set(stopwords.words("english")))
    tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
    tokens = tokens_re.findall(s)
    stems = [ps.stem(t) for t in tokens if t not in stop_words]
    return stems

#bag of words
def create_bag(dataframe):
    corpus = []
    for i in range(len(dataframe)):
        corpus.append((str(dataframe.iloc[i].title) + " " + str(dataframe.iloc[i].text)))
    vectorizer = CountVectorizer(max_df=0.5, tokenizer=tokenize)
    # creation du vocabulaire
    vector = vectorizer.fit_transform(corpus)
    df = pd.DataFrame(data=vectorizer.transform(corpus).toarray(), columns=vectorizer.get_feature_names_out())
    df.insert(0, "our_rating", dataframe["our rating"])
    return (vectorizer, df)
    
#appel de tout
def process(data):
  
  # 1 : suppression doublons 
  data = cleaningData(data)

  # 2 : bag of words
  bag, df = create_bag(data)

  #3 : return bag and data
  return(bag, df)

# Cellule execution:



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

df1 = pd.read_csv('data/HAI817_Projet_train.csv')
df2 = pd.read_csv('data/HAI817_Projet_test.csv')

bag_train, df_train = process(df1)

bag_test, df_test = process(df2)

# print(len(bag_test.vocabulary_))
# print(len(bag_train.vocabulary_))

df_1, df_2 = intersect(bag_train, bag_test, df_train, df_test)

# df_1 = df_train
# df_2 = df_test

In [ ]:
df_train_bin = df_1[(df_1['our_rating'] == 'true') | (df_1['our_rating'] == 'false')]
df_test_bin = df_2[(df_2['our_rating'] == 'true') | (df_2['our_rating'] == 'false')]


# print(df_train_bin['our_rating'].values_count())

X_train = df_train_bin.values[:, 1:]
y_train = df_train_bin.values[:, 0]

X_test = df_test_bin.values[:, 1:]
y_test = df_test_bin.values[:, 0]



In [ ]:
from sklearn.metrics import accuracy_score

# choix du classifieur
clf = LogisticRegression(max_iter=1500)

# entrainement
clf.fit(X_train, y_train)

# application
res = clf.predict(X_test)

print(f'accuracy = {accuracy_score(res, y_test)}')


In [ ]:
bag_train.head()

In [ ]:
display(bag_train)

In [ ]:
bag_train.to_csv("bow.csv")

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [ ]:
X = bag_train.values[:, 1:]
y = bag_train.values[:, 0]


In [ ]:

models = []
models.append(('LR', LogisticRegression(max_iter=3000)))
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVC', SVC(gamma='auto')))
models.append(('LSVC', LinearSVC()))
models.append(('RFC', RandomForestClassifier()))

seed = 7
results = []
names = []
scoring='accuracy'
for name,model in models:
    kfold = KFold(n_splits=10, random_state=seed,shuffle=True)
    cv_results = cross_val_score(model, X, y, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg) 


In [ ]:
clf = LogisticRegression(max_iter=3000)
clf.fit(X, y)
res = clf.score(X, y)
print(res)